# 原子体系建模

In [1]:
import numpy as np
from scipy.spatial import distance
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  #3D plot

经验势$E_{\mathrm{emp}}=V(r_1,r_2)$和切割半径$R_{\mathrm{cut}}$表示函数

In [2]:
def emperical_potential(r1,r2,a=5,b=10):
    d = distance.euclidean(r1,r2)
    return a / (d ** 12) - b / (d ** 6)

In [3]:
def cutoff_func(r_ij,r_c = 3.0):  #f_c(R_ij)
    if r_ij > r_c:
        return 0
    else:
        return 0.5*(np.cos(np.pi * r_ij / r_c) + 1)

原子类,包含原子坐标

In [4]:
class Atom():
    def __init__(self,coords):
        self.coords = coords
        self.individual_potential = 0
        self.g1_value = 0  #径向对称函数
        self.g2_value = 0  #径向对称函数
        self.g3_value = 0  #径向对称函数
    def individual_potential_cal(self,neighbors):
        for neighbor in neighbors:  #neighbors为相对距离列表
            self.individual_potential += 0.5 * emperical_potential(0,neighbor)
    def g1_cal(self,neighbors):
        for neighbor in neighbors:
            self.g1_value += cutoff_func(neighbor)
    def g2_cal(self,neighbors,eta = 0.1,r_s = 0.8):
        for neighbor in neighbors:
            self.g2_value += np.exp(-eta * (neighbor-r_s)**2)*cutoff_func(neighbor)
    def g3_cal(self,neighbors,kappa = 0.4):
        for neighbor in neighbors:
            self.g3_value += np.cos(kappa*neighbor)*cutoff_func(neighbor)

原子点集类,含维数、总势能、原子个数、坐标范围等参数,并有随机生成、输入列表生成、势能计算等方法

In [5]:
class AtomCollection():
    def __init__(self):
        """
        Args:
        coords_range: suppose (x, ...) is the coordinate of an atom while range is (a,b), then a<x<b, ...
        """
        self.atoms = []
        self.num = 0
        self.dimension = 0
        self.potential = 0
        self.coords_range=(-5,5)
        
    def random_generate(self,size,dimension=2):
        """generate a collection of atoms by using random numbers
        
        Args:
            size: the number of atoms in self.atoms
            dimension: 2 or 3
        """
        self.dimension = dimension
        temp_coords = np.random.uniform(self.coords_range[0],self.coords_range[1],
                                        dimension).tolist()  #1*dimension shape-like list
        self.num += 1
        self.atoms.append(Atom(temp_coords))
        while self.num < size:
            flag = True
            temp_coords = np.random.uniform(self.coords_range[0],self.coords_range[1],
                                            dimension).tolist()
            for atom in self.atoms:
                if distance.euclidean(atom.coords,temp_coords) < 0.1:  #太近了先不考虑
                    flag = False
                    break
            if flag is True:
                self.num +=1
                self.atoms.append(Atom(temp_coords))
                    
    def list_generate(self,coords_list):
        """generate a collection of atoms by using a given list
        
        Args:
            coords_list: shape like [[1,1,...],[2,2,...],...]
        """
        if coords_list == []:
            return
        for coords in coords_list:
            self.atoms.append(Atom(coords))
        self.num = len(coords_list)
        self.dimension = len(coords_list[0])
    
    def distance_cal(self,center):  #计算体系中其他所有原子和中心原子的相对距离
        dis = []
        for atom in self.atoms:
            if atom is not center:
                dis.append(distance.euclidean(atom.coords,center.coords))
        return dis
    
    def potential_cal(self):
        """calculate the total potential of the atom collection"""        
        for atom in self.atoms: #将每个原子的势能累加得到体系总势能
            self.potential += atom.individual_potential
        
    def plot_atoms(self):
        """plot all the atoms on one figure"""
        fig = plt.figure()
        
        if self.dimension == 2:
            fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
            ax = fig.add_subplot(111, aspect='equal', autoscale_on=False,
                         xlim=self.coords_range, ylim=self.coords_range)
            for atom in self.atoms:
                ax.scatter(atom.coords[0],atom.coords[1], color = 'black')
            
        if self.dimension == 3:
            ax = Axes3D(fig)
            for atom in self.atoms:
                ax.scatter(atom.coords[0], atom.coords[1],atom.coords[2])
            ax.set_zlabel('Z', fontdict={'size': 15, 'color': 'black'})
            ax.set_ylabel('Y', fontdict={'size': 15, 'color': 'black'})
            ax.set_xlabel('X', fontdict={'size': 15, 'color': 'black'})
    
    def clear_collection(self):
        """erase all the coordinates"""
        self.atoms = []
        self.num = 0
        self.dimension = 0
        self.potential = 0
        self.coords_range=(-5,5)

# 神经网络训练

In [6]:
import tensorflow as tf

数据整理

In [7]:
def data_reshape(atom_collection_set):
    """reshape the collections, with output like [[g11,g12,g13],[g21,g22,g23],...] & [E1,E2,...]"""
    g_factors = []
    potentials = []
    for collection in atom_collection_set:
        for atom in collection.atoms:
            #g_factors.append([atom.g1_value,atom.g2_value])
            g_factors.append([atom.g1_value,atom.g2_value,atom.g3_value])
            potentials.append(atom.individual_potential)
            #print(atom.individual_potential)
            #print(len(potentials))
    return np.array(g_factors),np.array(potentials)

In [8]:
def get_batch(params,energies,batch_size,start_index = 0):
    return params[start_index:start_index + batch_size],energies[start_index:start_index + batch_size]

网络模型

In [9]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1_g1 = tf.keras.layers.Dense(units=100)
        self.dense1_g2 = tf.keras.layers.Dense(units=100)
        self.dense1_g3 = tf.keras.layers.Dense(units=100)
        self.dense2_g1 = tf.keras.layers.Dense(units=20,activation = tf.nn.tanh)
        self.dense2_g2 = tf.keras.layers.Dense(units=20,activation = tf.nn.tanh)
        self.dense2_g3 = tf.keras.layers.Dense(units=20,activation = tf.nn.tanh)
        self.dense3_g1 = tf.keras.layers.Dense(units=1)
        self.dense3_g2 = tf.keras.layers.Dense(units=1)
        self.dense3_g3 = tf.keras.layers.Dense(units=1)

    def call(self, inputs):
        g1 = tf.slice(inputs,[0,0],[-1,1])  #tf.slice(data,begin,size)
        g1 = tf.reshape(g1,[-1,1])
        g2 = tf.slice(inputs,[0,1],[-1,1])
        g2 = tf.reshape(g2,[-1,1])
        g3 = tf.slice(inputs,[0,2],[-1,1])
        g3 = tf.reshape(g3,[-1,1])
        g1 = self.dense1_g1(g1)
        g1 = self.dense2_g1(g1)
        g1 = self.dense3_g1(g1)
        g2 = self.dense1_g2(g2)
        g2 = self.dense2_g2(g2)
        g2 = self.dense3_g2(g2)
        #g3 = self.dense1_g3(g3)
        #g3 = self.dense2_g3(g3)
        #g3 = self.dense3_g3(g3)
        #output = g1 + g2 + g3
        output = g1 + g2
        return output

超参数hyperparameters

In [27]:
#network params
num_epoch = 1.0
batch_size = 10
rate = 0.00000001

#configurations generation params
atom_size = 50
configurations = 400
dim = 3

#cross validation params
train_ratio = 0.9
test_ratio = 0.1
split = 10
seed = 1  #不要每次运行程序cv结果一样就seed = None

建数据集

In [11]:
config = 0
train_collections = []
r_ij = []
          
while config < configurations:
    temp = AtomCollection()
    temp.random_generate(size = atom_size,dimension = dim)
    for atom in temp.atoms:
        r_ij = temp.distance_cal(atom)
        atom.individual_potential_cal(r_ij)
        atom.g1_cal(r_ij)
        atom.g2_cal(r_ij)
        atom.g3_cal(r_ij)
        r_ij = []
    train_collections.append(temp)
    config += 1

In [12]:
g_factors,potentials = data_reshape(train_collections)
#len(train_collections)
#g_factors[:15]
potentials[:15]

array([-0.57265367, -0.01792639, -0.59706773, -5.70994758, -0.73442609,
       -0.55097378, -0.34194468, -0.20500102, -2.67012234, -0.00751078,
       -0.04067032, -0.12556378, -0.00934528, -0.21865232, -0.22724583])

训练及测试评估

In [16]:
from sklearn.model_selection import ShuffleSplit

In [28]:
model = MLP()
optimizer = tf.keras.optimizers.Adam(learning_rate=rate)
num_batches = int(len(potentials)*train_ratio//batch_size*num_epoch)
rs = ShuffleSplit(n_splits=split,test_size=test_ratio,
                  train_size=train_ratio,random_state=seed)  #划分数据集为训练集和测试集
times = 1

In [29]:
for train_index,test_index in rs.split(potentials):
    print("times {}".format(times))
    for batch_index in range(num_batches):
        X_train,y_train = get_batch(g_factors[train_index],potentials[train_index],batch_size,
                                    batch_size * batch_index)  #X: (batch_size,num_of_factors), y: (batch_size,)
        X_train = tf.convert_to_tensor(X_train,dtype = 'float32')
        with tf.GradientTape() as tape:
            y_pred = model(X_train)
            y_temp = tf.convert_to_tensor(y_train.reshape(-1,1),dtype = 'float32')
            loss = 0.5 * tf.reduce_sum(tf.square(y_pred - y_temp))
            if batch_index % 100 == 0:
                print("batch {}: loss {}".format(batch_index,loss.numpy()))
        grad = tape.gradient(loss,model.variables)  #model.variables直接调用模型变量
        optimizer.apply_gradients(grads_and_vars=zip(grad,model.variables))
    
    X_test,y_test = g_factors[test_index],potentials[test_index]
    y_pred_test = model.predict(X_test)
    error = (y_pred_test - y_test)/y_test
    print("average_error {}".format(np.mean(error)))

    times += 1
    print("\n")

times 1
batch 0: loss 5.49675989151001
batch 100: loss 3.8996901512145996
batch 200: loss 3.3908774852752686
batch 300: loss 2574467.25
batch 400: loss 1319174.875
batch 500: loss 1.502172589302063
batch 600: loss 5.0454230308532715
batch 700: loss 12.886371612548828
batch 800: loss 664905252864.0
batch 900: loss 154.4141387939453
batch 1000: loss 28925.044921875
batch 1100: loss 2.557191101035315e+16
batch 1200: loss 6.196959018707275
batch 1300: loss 4.3756608963012695
batch 1400: loss 2.7315046787261963
batch 1500: loss 21.66996192932129
batch 1600: loss 11.284544944763184
batch 1700: loss 77180.7734375
average_error -0.10943256786580177


times 2
batch 0: loss 272.8937072753906
batch 100: loss 161775872.0
batch 200: loss 1732871.75
batch 300: loss 803.5100708007812
batch 400: loss 7411123200.0
batch 500: loss 1354637.625
batch 600: loss 2678.078857421875
batch 700: loss 32959.359375
batch 800: loss 521048.28125
batch 900: loss 32710.408203125
batch 1000: loss 764.681884765625
batch